In [1]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

--2025-05-20 21:53:46--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.64.207, 74.125.126.207, 142.251.184.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.64.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   301MB/s    in 6.5s    

2025-05-20 21:53:53 (295 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [2]:
!tar -xf dakshina_dataset_v1.0.tar

In [3]:
import wandb

wandb.login(key="93b4881869bab13360839595daa56e51dd0405df")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivam-da24m018 (shivam-da24m018-iitmaana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import os
import tensorflow as tf
import wandb
import argparse
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
import time

# Set random seeds for reproducibility
def set_random_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_random_seeds()

# Check for GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)
else:
    print("Using CPU")

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)


2025-05-20 21:54:13.804929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747778053.997865      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747778054.057877      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using 2 GPU(s)


In [5]:
# Copy the code from config.py
DEFAULT_CONFIG = {
    'cell_type': 'GRU',
    'embed_size': 128,
    'hidden_size': 256,
    'encoder_layers': 1,
    'dropout': 0.2,
    'batch_size': 64,
    'learning_rate': 0.0005,
    'teacher_forcing_ratio': 0.5,
    'attention_type': 'dot',
    'clip_norm': 5.0,
    'epochs': 1
}

def calculate_model_complexity(embedding_size, hidden_size, sequence_length, vocabulary_size):
    """
    Calculate model complexity
    Parameters:
        embedding_size: Size of embedding vectors
        hidden_size: Size of hidden state
        sequence_length: Maximum sequence length
        vocabulary_size: Size of vocabulary
    """
    # Embedding layer parameters
    embedding_params = 2 * vocabulary_size * embedding_size  # Source and target embeddings
    
    # RNN parameters (for both encoder and decoder)
    encoder_rnn_params = embedding_size * hidden_size + hidden_size * hidden_size + hidden_size  # Input-to-hidden, hidden-to-hidden, bias
    decoder_rnn_params = embedding_size * hidden_size + hidden_size * hidden_size + hidden_size
    
    # Output layer parameters
    output_params = hidden_size * vocabulary_size + vocabulary_size  # Hidden-to-output, bias
    
    # Total parameters
    total_params = embedding_params + encoder_rnn_params + decoder_rnn_params + output_params
    
    # Computations
    # Encoder computations
    encoder_comp = sequence_length * (embedding_size * hidden_size + hidden_size * hidden_size)
    
    # Decoder computations
    decoder_comp = sequence_length * (embedding_size * hidden_size + hidden_size * hidden_size + hidden_size * vocabulary_size)
    
    # Total computations
    total_comp = encoder_comp + decoder_comp
    
    return {
        'embedding_params': embedding_params,
        'encoder_rnn_params': encoder_rnn_params,
        'decoder_rnn_params': decoder_rnn_params,
        'output_params': output_params,
        'total_params': total_params,
        'encoder_comp': encoder_comp,
        'decoder_comp': decoder_comp,
        'total_comp': total_comp
    }


In [6]:
# Copy the code from data_utils.py
class TransliterationDataset(tf.keras.utils.Sequence):
    def __init__(self, source_tensor, target_tensor, batch_size=64, shuffle=True):
        self.source_tensor = source_tensor
        self.target_tensor = target_tensor
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.source_tensor))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.source_tensor) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        return self.source_tensor[batch_indices], self.target_tensor[batch_indices]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

def load_corpus(filepath):
    """Extract transliteration pairs from corpus file"""
    corpus_entries = []
    with open(filepath, 'r', encoding='utf-8') as corpus_file:
        for entry in corpus_file:
            segments = entry.strip().split('\t')
            if len(segments) >= 2:
                roman_text = segments[1]
                native_text = segments[0]
                corpus_entries.append((roman_text, native_text))
    return corpus_entries

def create_vocabulary(corpus_entries):
    """Generate character-to-index mappings from corpus"""
    roman_character_set = set()
    native_character_set = set()
    
    for roman_text, native_text in corpus_entries:
        roman_character_set.update(roman_text)
        native_character_set.update(native_text)
    
    print("Sample native script chars:", sorted(native_character_set)[:10])
    print("Sample roman script chars:", sorted(roman_character_set)[:10])
    
    # Add special tokens
    special_tokens = ['<pad>', '<sos>', '<eos>']
    for token in special_tokens:
        roman_character_set.add(token)
        native_character_set.add(token)
    
    # Create dictionaries
    roman_to_idx = {char: idx for idx, char in enumerate(sorted(roman_character_set))}
    idx_to_roman = {idx: char for char, idx in roman_to_idx.items()}
    
    native_to_idx = {char: idx for idx, char in enumerate(sorted(native_character_set))}
    idx_to_native = {idx: char for char, idx in native_to_idx.items()}
    
    return roman_to_idx, idx_to_roman, native_to_idx, idx_to_native

def encode_sequences(corpus_entries, roman_to_idx, native_to_idx, max_roman_len=None, max_native_len=None):
    """Convert character sequences to index sequences"""
    if max_roman_len is None:
        max_roman_len = max(len(roman) for roman, _ in corpus_entries) + 2  # +2 for <sos> and <eos>
    
    if max_native_len is None:
        max_native_len = max(len(native) for _, native in corpus_entries) + 2
    
    roman_sequences = []
    native_sequences = []
    
    for roman_text, native_text in corpus_entries:
        # Process roman text
        roman_indices = [roman_to_idx['<sos>']]
        roman_indices.extend(roman_to_idx[char] for char in roman_text)
        roman_indices.append(roman_to_idx['<eos>'])
        
        # Pad roman sequence
        padding_length = max_roman_len - len(roman_indices)
        roman_indices.extend([roman_to_idx['<pad>']] * padding_length)
        roman_sequences.append(roman_indices)
        
        # Process native text
        native_indices = [native_to_idx['<sos>']]
        native_indices.extend(native_to_idx[char] for char in native_text)
        native_indices.append(native_to_idx['<eos>'])
        
        # Pad native sequence
        padding_length = max_native_len - len(native_indices)
        native_indices.extend([native_to_idx['<pad>']] * padding_length)
        native_sequences.append(native_indices)
    
    return np.array(roman_sequences), np.array(native_sequences), max_roman_len, max_native_len

def prepare_data_pipeline(corpus_dir, batch_size=64):
    """Setup complete data pipeline for training and evaluation"""
    train_corpus = load_corpus(os.path.join(corpus_dir, 'hi.translit.sampled.train.tsv'))
    dev_corpus = load_corpus(os.path.join(corpus_dir, 'hi.translit.sampled.dev.tsv'))
    test_corpus = load_corpus(os.path.join(corpus_dir, 'hi.translit.sampled.test.tsv'))
    
    # Build character mappings from training and validation data
    roman_to_idx, idx_to_roman, native_to_idx, idx_to_native = create_vocabulary(train_corpus + dev_corpus)
    
    # Transform data to indices
    X_train, y_train, max_roman_len, max_native_len = encode_sequences(
        train_corpus, roman_to_idx, native_to_idx)
    
    X_dev, y_dev, _, _ = encode_sequences(
        dev_corpus, roman_to_idx, native_to_idx, max_roman_len, max_native_len)
    
    X_test, y_test, _, _ = encode_sequences(
        test_corpus, roman_to_idx, native_to_idx, max_roman_len, max_native_len)
    
    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_dev, y_dev)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)
    
    return {
        'train_dataset': train_dataset,
        'val_dataset': val_dataset,
        'test_dataset': test_dataset,
        'roman_to_idx': roman_to_idx,
        'idx_to_roman': idx_to_roman,
        'native_to_idx': native_to_idx,
        'idx_to_native': idx_to_native,
        'max_roman_len': max_roman_len,
        'max_native_len': max_native_len
    }


In [7]:
# Copy the code from model.py
class AttentionMechanism(tf.keras.layers.Layer):
    def __init__(self, units, method='dot'):
        super(AttentionMechanism, self).__init__()
        self.units = units
        self.method = method.lower()
        
        if self.method == 'bahdanau':
            # Bahdanau attention (additive)
            self.W1 = tf.keras.layers.Dense(units)
            self.W2 = tf.keras.layers.Dense(units)
            self.V = tf.keras.layers.Dense(1)
        # No parameters needed for Luong attention (multiplicative)
    
    def call(self, query, values):
        # query shape: [batch_size, hidden_size]
        # values shape: [batch_size, seq_len, hidden_size]
        
        if self.method == 'bahdanau':
            # Bahdanau attention
            # Expand query to [batch_size, 1, hidden_size]
            query_expanded = tf.expand_dims(query, 1)
            
            # Calculate score
            score = self.V(tf.nn.tanh(self.W1(query_expanded) + self.W2(values)))
            
            # Remove the last dimension
            score = tf.squeeze(score, axis=-1)
        else:
            # Luong attention (multiplicative)
            # Reshape query to [batch_size, hidden_size, 1]
            query_reshaped = tf.expand_dims(query, -1)
            
            # Calculate score: [batch_size, seq_len, 1]
            score = tf.matmul(values, query_reshaped)
            
            # Remove the last dimension
            score = tf.squeeze(score, axis=-1)
        
        # Apply softmax to get attention weights
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # Calculate context vector
        context = tf.matmul(tf.expand_dims(attention_weights, 1), values)
        context = tf.squeeze(context, axis=1)
        
        return context, attention_weights

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units, cell_type='gru', num_layers=1, dropout_rate=0.0):
        super(Encoder, self).__init__()
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.cell_type = cell_type.lower()
        
        # Embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        # RNN layers
        self.rnn_cells = []
        for i in range(num_layers):
            if self.cell_type == 'lstm':
                cell = tf.keras.layers.LSTM(
                    hidden_units,
                    return_sequences=True,
                    return_state=True,
                    dropout=dropout_rate if i < num_layers-1 else 0.0,
                    recurrent_initializer='glorot_uniform'
                )
            elif self.cell_type == 'gru':
                cell = tf.keras.layers.GRU(
                    hidden_units,
                    return_sequences=True,
                    return_state=True,
                    dropout=dropout_rate if i < num_layers-1 else 0.0,
                    recurrent_initializer='glorot_uniform'
                )
            else:  # Simple RNN
                cell = tf.keras.layers.SimpleRNN(
                    hidden_units,
                    return_sequences=True,
                    return_state=True,
                    dropout=dropout_rate if i < num_layers-1 else 0.0,
                    recurrent_initializer='glorot_uniform'
                )
            self.rnn_cells.append(cell)
    
    def call(self, x, training=False):
        # x shape: [batch_size, seq_len]
        
        # Embedding
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        
        # Process through RNN layers
        states = []
        for i, rnn_cell in enumerate(self.rnn_cells):
            if self.cell_type == 'lstm':
                x, state_h, state_c = rnn_cell(x, training=training)
                states.append([state_h, state_c])
            else:
                x, state = rnn_cell(x, training=training)
                states.append(state)
        
        return x, states

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units, cell_type='gru',
                 num_layers=1, dropout_rate=0.0, attention_type='dot'):
        super(Decoder, self).__init__()
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.cell_type = cell_type.lower()
        self.vocab_size = vocab_size
        
        # Embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        # Attention mechanism
        self.attention = AttentionMechanism(hidden_units, attention_type)
        
        # RNN layers
        self.rnn_cells = []
        for i in range(num_layers):
            if self.cell_type == 'lstm':
                cell = tf.keras.layers.LSTM(
                    hidden_units,
                    return_sequences=True,
                    return_state=True,
                    dropout=dropout_rate if i < num_layers-1 else 0.0,
                    recurrent_initializer='glorot_uniform'
                )
            elif self.cell_type == 'gru':
                cell = tf.keras.layers.GRU(
                    hidden_units,
                    return_sequences=True,
                    return_state=True,
                    dropout=dropout_rate if i < num_layers-1 else 0.0,
                    recurrent_initializer='glorot_uniform'
                )
            else:  # Simple RNN
                cell = tf.keras.layers.SimpleRNN(
                    hidden_units,
                    return_sequences=True,
                    return_state=True,
                    dropout=dropout_rate if i < num_layers-1 else 0.0,
                    recurrent_initializer='glorot_uniform'
                )
            self.rnn_cells.append(cell)
        
        # Output layer
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # Dropout
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, hidden, encoder_output, training=False):
        # x shape: [batch_size]
        # hidden shape: list of states for each layer
        # encoder_output shape: [batch_size, seq_len, hidden_units]
        
        # Get the last hidden state
        if self.cell_type == 'lstm':
            query = hidden[-1][0]  # Use hidden state, not cell state
        else:
            query = hidden[-1]
        
        # Calculate attention
        context, attention_weights = self.attention(query, encoder_output)
        
        # Expand x to [batch_size, 1]
        x = tf.expand_dims(x, 1)
        
        # Embedding
        x = self.embedding(x)  # [batch_size, 1, embedding_dim]
        
        # Concatenate context vector and embedding
        x = tf.concat([tf.expand_dims(context, 1), x], axis=-1)
        
        # Apply dropout
        x = self.dropout(x, training=training)
        
        # Process through RNN layers
        new_hidden = []
        for i, rnn_cell in enumerate(self.rnn_cells):
            if i > 0:
                x = self.dropout(x, training=training)
            
            if self.cell_type == 'lstm':
                current_hidden = hidden[i]
                x, state_h, state_c = rnn_cell(x, initial_state=current_hidden, training=training)
                new_hidden.append([state_h, state_c])
            else:
                current_hidden = hidden[i]
                x, state = rnn_cell(x, initial_state=current_hidden, training=training)
                new_hidden.append(state)
        
        # Output
        x = self.fc(tf.concat([x[:, 0], context], axis=-1))
        
        return x, new_hidden, attention_weights

class Seq2SeqModel(tf.keras.Model):
    def __init__(self, encoder, decoder, start_token_idx, end_token_idx):
        super(Seq2SeqModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
    
    def build(self, input_shape):
    # This method explicitly builds the model
    # input_shape is a tuple of (source_shape, target_shape)
        source_shape, target_shape = input_shape
        
        # Build encoder
        self.encoder.build(source_shape)
        
        # Build decoder - create a dummy state to help build it
        dummy_source = tf.zeros((1,) + source_shape[1:], dtype=tf.int32)
        dummy_encoder_output, dummy_states = self.encoder(dummy_source)
        
        # Build decoder with dummy inputs
        dummy_decoder_input = tf.zeros((1,), dtype=tf.int32)
        
        # Check if we're using LSTM (which returns a list of [state_h, state_c] for each layer)
        if isinstance(dummy_states[0], list):
            # For LSTM, use the hidden state (state_h)
            state_shape = dummy_states[0][0].shape
        else:
            # For GRU or RNN, use the state directly
            state_shape = dummy_states[0].shape
            
        self.decoder.build([(1,), state_shape, dummy_encoder_output.shape])
    
        self.built = True

    
    def call(self, inputs, training=False):
        # Unpack inputs
        source, target = inputs
        batch_size = tf.shape(source)[0]
        target_length = tf.shape(target)[1]
        
        # Encode the source sequence
        encoder_output, encoder_states = self.encoder(source, training=training)
        
        # Initialize output tensor - make sure it has the right shape
        outputs = tf.TensorArray(tf.float32, size=target_length-1)
        
        # Prepare decoder input and states
        decoder_input = tf.fill([batch_size], self.start_token_idx)
        decoder_states = encoder_states
        
        # Teacher forcing ratio
        teacher_forcing_ratio = 0.5 if training else 0.0
        
        # Decode step by step - start from position 1 (after start token)
        for t in range(1, target_length):
            # Get output from decoder
            prediction, decoder_states, _ = self.decoder(
                decoder_input, decoder_states, encoder_output, training=training)
            
            # Store prediction
            outputs = outputs.write(t-1, prediction)
            
            # Teacher forcing
            if training and tf.random.uniform([]) < teacher_forcing_ratio:
                decoder_input = target[:, t]
            else:
                decoder_input = tf.argmax(prediction, axis=1)
        
        # Stack outputs - ensure it has the right shape to match target
        outputs = tf.transpose(outputs.stack(), [1, 0, 2])
        
        return outputs
    
    def translate(self, source, max_length=50):
        """
        Translate source sequences to target sequences
        Args:
            source: Source sequence tensor [batch_size, seq_len]
            max_length: Maximum length of generated sequence
        Returns:
            translations: Generated sequences [batch_size, seq_len]
            attention_weights: Attention weights for visualization [batch_size, tgt_len, src_len]
        """
        batch_size = tf.shape(source)[0]
        
        # Encode the source sequence
        encoder_output, encoder_states = self.encoder(source, training=False)
        
        # Prepare decoder input and states
        decoder_input = tf.fill([batch_size], self.start_token_idx)
        decoder_states = encoder_states
        
        # Initialize result tensors - specify int32 dtype
        translations = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
        attention_weights_array = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
        
        # Store first token (start token)
        translations = translations.write(0, decoder_input)
        
        for t in range(1, max_length):
            # Get output from decoder
            prediction, decoder_states, attention_weights = self.decoder(
                decoder_input, decoder_states, encoder_output, training=False)
            
            # Get predicted token and cast to int32
            predicted_id = tf.argmax(prediction, axis=1)
            predicted_id = tf.cast(predicted_id, tf.int32)  # Cast to int32
            
            # Store results
            translations = translations.write(t, predicted_id)
            attention_weights_array = attention_weights_array.write(t-1, attention_weights)
            
            # Break if end token is predicted for all sequences in batch
            if tf.reduce_all(tf.equal(predicted_id, self.end_token_idx)):
                break
            
            # Use predicted token as next input
            decoder_input = predicted_id
        
        # Stack results
        translations = tf.transpose(translations.stack())  # [batch_size, seq_len]
        attention_weights = tf.transpose(attention_weights_array.stack(), [1, 0, 2])  # [batch_size, tgt_len, src_len]
        
        return translations, attention_weights

def create_model(source_vocab_size, target_vocab_size, embedding_dim=256, hidden_units=512,
                 cell_type='gru', num_layers=2, dropout_rate=0.1, attention_type='dot',
                 start_token_idx=0, end_token_idx=1):
    encoder = Encoder(
        source_vocab_size, embedding_dim, hidden_units,
        cell_type, num_layers, dropout_rate
    )
    
    decoder = Decoder(
        target_vocab_size, embedding_dim, hidden_units,
        cell_type, num_layers, dropout_rate, attention_type
    )
    
    model = Seq2SeqModel(encoder, decoder, start_token_idx, end_token_idx)
    
    return model


In [8]:
# Copy the code from train.py
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate):
        super(CustomSchedule, self).__init__()
        self.initial_learning_rate = initial_learning_rate
    
    def __call__(self, step):
        return self.initial_learning_rate

def train_step(model, source, target, optimizer, loss_function, clip_norm):
    with tf.GradientTape() as tape:
        # Forward pass
        predictions = model((source, target), training=True)
        
        # Get actual batch size from the current batch
        batch_size = tf.shape(source)[0]
        
        # Reshape for loss calculation
        target_seq = target[:, 1:]  # Exclude start token
        
        # Calculate loss
        loss = loss_function(target_seq, predictions)
    
    # Calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # Clip gradients
    gradients, _ = tf.clip_by_global_norm(gradients, clip_norm)
    
    # Apply gradients
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return tf.reduce_mean(loss)  # Return scalar loss

def validation_step(model, source, target, loss_function):
    """Evaluate model on validation data"""
    # Forward pass (no teacher forcing)
    predictions = model((source, target), training=False)
    
    # Reshape for loss calculation
    target_seq = target[:, 1:]  # Exclude start token
    predictions_seq = predictions  # Already aligned in model
    
    # Calculate loss
    mask = tf.math.logical_not(tf.math.equal(target_seq, 0))  # 0 is pad token
    mask = tf.cast(mask, dtype=tf.float32)
    loss = loss_function(target_seq, predictions_seq, sample_weight=mask)
    
    return loss

def compute_accuracy(model, dataset, target_to_idx, idx_to_target, idx_to_source):
    """Calculate word-level accuracy"""
    correct_predictions = 0
    total_samples = 0
    prediction_details = []
    
    # Find special tokens
    start_token_idx = target_to_idx['<sos>']
    end_token_idx = target_to_idx['<eos>']
    pad_token_idx = target_to_idx['<pad>']
    
    for source_batch, target_batch in tqdm(dataset, desc="Computing accuracy"):
        batch_size = tf.shape(source_batch)[0]
        
        # Translate
        translations, attention_weights = model.translate(source_batch)
        
        # Process each example in batch
        for i in range(batch_size):
            # Extract source sequence
            source_seq = []
            for j in range(tf.shape(source_batch)[1]):
                idx = source_batch[i, j].numpy()
                if idx in idx_to_source and idx_to_source[idx] not in ['<pad>', '<sos>', '<eos>']:
                    source_seq.append(idx_to_source[idx])
            
            # Extract prediction and target sequences
            pred_seq = []
            target_seq = []
            
            # Process prediction
            for j in range(tf.shape(translations)[1]):
                pred_idx = translations[i, j].numpy()
                if pred_idx == end_token_idx or pred_idx == pad_token_idx:
                    break
                if pred_idx in idx_to_target and idx_to_target[pred_idx] not in ['<pad>', '<sos>', '<eos>']:
                    pred_seq.append(idx_to_target[pred_idx])
            
            # Process target
            for j in range(1, tf.shape(target_batch)[1]):  # Skip start token
                target_idx = target_batch[i, j].numpy()
                if target_idx == end_token_idx or target_idx == pad_token_idx:
                    break
                if target_idx in idx_to_target and idx_to_target[target_idx] not in ['<pad>', '<sos>', '<eos>']:
                    target_seq.append(idx_to_target[target_idx])
            
            # Convert to strings
            pred_word = ''.join(pred_seq)
            target_word = ''.join(target_seq)
            source_word = ''.join(source_seq)
            
            # Get attention weights
            attn_matrix = attention_weights[i, :len(pred_seq), :len(source_seq)].numpy()
            
            # Store prediction details
            prediction_details.append({
                'source': source_word,
                'prediction': pred_word,
                'target': target_word,
                'attention': attn_matrix
            })
            
            # Update accuracy
            if pred_word == target_word:
                correct_predictions += 1
            total_samples += 1
    
    accuracy = correct_predictions / total_samples if total_samples > 0 else 0
    return accuracy, prediction_details

def train_model(model, data_loaders, config):
    # Setup optimizer and loss function
    optimizer = tf.keras.optimizers.Adam(config['learning_rate'])
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    best_accuracy = 0
    
    # Training loop
    for epoch in range(config['epochs']):
        total_loss = 0.0
        num_batches = 0
        
        for source_batch, target_batch in tqdm(data_loaders['train_dataset'], desc=f"Epoch {epoch+1} Training"):
            # Get loss for this batch
            batch_loss = train_step(
                model, source_batch, target_batch, optimizer, loss_function, config['clip_norm']
            )
            
            # Accumulate loss (use scalar value)
            total_loss += float(batch_loss)
            num_batches += 1
        
        # Calculate average loss
        train_loss = total_loss / num_batches
        
        # Validation phase
        val_loss = 0
        val_steps = 0
        
        for source_batch, target_batch in tqdm(data_loaders['val_dataset'], desc="Validation"):
            batch_loss = validation_step(
                model,
                source_batch,
                target_batch,
                loss_function
            )
            
            val_loss += batch_loss
            val_steps += 1
        
        val_loss = val_loss / val_steps
        
        # Calculate accuracy
        accuracy, _ = compute_accuracy(
            model,
            data_loaders['val_dataset'],
            data_loaders['native_to_idx'],
            data_loaders['idx_to_native'],
            data_loaders['idx_to_roman']
        )
        
        # Log metrics
        wandb.log({
            'epoch': epoch,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': accuracy,
        })
        
        # Print progress
        print(f'\tTraining Loss: {train_loss:.3f}')
        print(f'\tValidation Loss: {val_loss:.3f}')
        print(f'\tAccuracy: {accuracy:.4f}')
        
        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            model.save_weights('models/best-transliteration-model.weights.h5')
            print(f'\tNew best accuracy: {best_accuracy:.4f}, model saved')
    
    return best_accuracy


In [9]:
# Copy the code from test.py
def evaluate_model(model, data_loaders):
    """Evaluate model on test set and analyze results"""
    # Load best model weights
    model.load_weights('models/best-transliteration-model.weights.h5')
    
    # Calculate accuracy and get predictions
    test_accuracy, prediction_results = compute_test_accuracy(
        model,
        data_loaders['test_dataset'],
        data_loaders['native_to_idx'],
        data_loaders['idx_to_native'],
        data_loaders['idx_to_roman']
    )
    
    # Calculate character-level accuracy
    char_accuracy = compute_character_accuracy(prediction_results)
    
    # Analyze error patterns
    error_patterns = analyze_error_patterns(prediction_results)
    
    # Log metrics to wandb
    wandb.log({
        'test_word_accuracy': test_accuracy,
        'test_char_accuracy': char_accuracy
    })
    
    # Print results
    print(f'Test Word Accuracy: {test_accuracy:.4f}')
    print(f'Test Character Accuracy: {char_accuracy:.4f}')
    
    # Print most common errors
    print("\nMost common error patterns:")
    for (target_char, pred_char), count in sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"Target '{target_char}' predicted as '{pred_char}': {count} times")
    
    # Log sample predictions
    sample_entries = []
    for i, pred in enumerate(prediction_results[:10]):
        sample_entries.append([i, pred['prediction'], pred['target'], pred['prediction'] == pred['target']])
    
    wandb.log({
        "prediction_samples": wandb.Table(
            columns=["Index", "Prediction", "Target", "Correct"],
            data=sample_entries
        )
    })
    
    # Create error analysis visualization
    top_errors = sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)[:10]
    error_labels = [f"{t}->{p}" for (t, p), _ in top_errors]
    error_counts = [count for _, count in top_errors]
    
    plt.figure(figsize=(10, 6))
    plt.bar(error_labels, error_counts)
    plt.xlabel('Error Type (Target->Prediction)')
    plt.ylabel('Count')
    plt.title('Top 10 Error Patterns')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Log visualization
    wandb.log({"error_analysis": wandb.Image(plt)})
    plt.close()
    
    return test_accuracy, prediction_results

def compute_test_accuracy(model, dataset, target_to_idx, idx_to_target, idx_to_source):
    """Calculate accuracy on test set and collect detailed predictions"""
    correct_count = 0
    total_count = 0
    all_predictions = []
    
    # Find special tokens
    start_token_idx = target_to_idx['<sos>']
    end_token_idx = target_to_idx['<eos>']
    pad_token_idx = target_to_idx['<pad>']
    
    for source_batch, target_batch in tqdm(dataset, desc="Testing"):
        batch_size = tf.shape(source_batch)[0]
        
        # Translate
        translations, attention_weights = model.translate(source_batch)
        
        # Process each example in batch
        for i in range(batch_size):
            # Extract source sequence
            source_seq = []
            for j in range(tf.shape(source_batch)[1]):
                idx = source_batch[i, j].numpy()
                if idx in idx_to_source and idx_to_source[idx] not in ['<pad>', '<sos>', '<eos>']:
                    source_seq.append(idx_to_source[idx])
            
            # Extract prediction and target sequences
            pred_seq = []
            target_seq = []
            
            # Process prediction
            for j in range(tf.shape(translations)[1]):
                pred_idx = translations[i, j].numpy()
                if pred_idx == end_token_idx or pred_idx == pad_token_idx:
                    break
                if pred_idx in idx_to_target and idx_to_target[pred_idx] not in ['<pad>', '<sos>', '<eos>']:
                    pred_seq.append(idx_to_target[pred_idx])
            
            # Process target
            for j in range(1, tf.shape(target_batch)[1]):  # Skip start token
                target_idx = target_batch[i, j].numpy()
                if target_idx == end_token_idx or target_idx == pad_token_idx:
                    break
                if target_idx in idx_to_target and idx_to_target[target_idx] not in ['<pad>', '<sos>', '<eos>']:
                    target_seq.append(idx_to_target[target_idx])
            
            # Convert to strings
            pred_word = ''.join(pred_seq)
            target_word = ''.join(target_seq)
            source_word = ''.join(source_seq)
            
            # Get attention weights
            attn_matrix = attention_weights[i, :len(pred_seq), :len(source_seq)].numpy()
            
            # Store prediction details
            all_predictions.append({
                'source': source_word,
                'prediction': pred_word,
                'target': target_word,
                'attention': attn_matrix
            })
            
            # Update accuracy counters
            if pred_word == target_word:
                correct_count += 1
            total_count += 1
    
    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy, all_predictions

def compute_character_accuracy(predictions):
    """Calculate character-level accuracy"""
    total_chars = 0
    correct_chars = 0
    
    for pred in predictions:
        prediction = pred['prediction']
        target = pred['target']
        
        # Compare characters up to the length of the shorter string
        min_len = min(len(prediction), len(target))
        for i in range(min_len):
            if prediction[i] == target[i]:
                correct_chars += 1
            total_chars += 1
        
        # Count remaining characters in longer string as errors
        total_chars += abs(len(prediction) - len(target))
    
    char_accuracy = correct_chars / total_chars if total_chars > 0 else 0
    return char_accuracy

def analyze_error_patterns(predictions):
    """Analyze common error patterns in predictions"""
    error_counter = Counter()
    
    for pred in predictions:
        prediction = pred['prediction']
        target = pred['target']
        
        if prediction != target:
            # Find the first position where they differ
            min_len = min(len(prediction), len(target))
            for i in range(min_len):
                if prediction[i] != target[i]:
                    error_pair = (target[i], prediction[i])
                    error_counter[error_pair] += 1
                    break
            
            # Handle length differences
            if len(prediction) != len(target) and min_len > 0 and i == min_len - 1:
                if len(prediction) < len(target):
                    error_pair = (target[min_len], "MISSING")
                else:
                    error_pair = ("MISSING", prediction[min_len])
                error_counter[error_pair] += 1
    
    return error_counter


In [10]:
# Copy the code from visualize.py
def render_attention_heatmap(attention_matrix, source_text, predicted_text, target_text, output_path=None):
    """Create a visualization of attention weights"""
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    
    # Convert attention to numpy if it's a tensor
    if isinstance(attention_matrix, tf.Tensor):
        attention_matrix = attention_matrix.numpy()
    
    # Create heatmap
    sns.heatmap(attention_matrix, ax=ax, cmap='viridis', cbar=True)
    
    # Set axis labels
    ax.set_xticklabels([''] + list(source_text), rotation=90)
    ax.set_yticklabels([''] + list(predicted_text))
    
    # Set title
    ax.set_title(f"Target: {target_text}")
    
    # Adjust layout
    fig.tight_layout()
    
    # Save if path is provided
    if output_path:
        fig.savefig(output_path)
    
    return fig

def create_attention_visualization_grid(prediction_results, sample_count=10, output_path='results/attention_grid.png'):
    """Create a grid of attention visualizations"""
    # Limit to available examples
    sample_count = min(sample_count, len(prediction_results))
    
    # Calculate grid dimensions
    grid_rows = 5
    grid_cols = 2
    
    # Create figure
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols*4, grid_rows*4))
    
    for i, ax in enumerate(axes.flat):
        if i < sample_count:
            pred = prediction_results[i]
            
            # Get attention matrix
            attention = pred['attention']
            if isinstance(attention, tf.Tensor):
                attention = attention.numpy()
            
            # Plot heatmap
            sns.heatmap(attention, ax=ax, cmap='Blues', cbar=False)
            
            # Set tick positions and labels
            source_chars = [''] + list(pred['source'])
            prediction_chars = [''] + list(pred['prediction'])
            
            # Set x-ticks positions
            ax.set_xticks(np.arange(len(source_chars)))
            ax.set_xticklabels(source_chars, rotation=90)
            
            # Set y-ticks positions
            ax.set_yticks(np.arange(len(prediction_chars)))
            ax.set_yticklabels(prediction_chars)
            
            # Set title
            ax.set_title(f"Target: {pred['target']}", fontsize=10)
        else:
            ax.axis('off')
    
    plt.tight_layout()
    
    # Create directory if needed
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save figure
    plt.savefig(output_path, dpi=300)
    plt.close()
    
    return output_path

def visualize_character_mappings(model, data_loaders, sample_count=5, output_dir='results/character_mappings'):
    """Visualize how characters are mapped between source and target"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Get a batch of examples
    for source_batch, target_batch in data_loaders['test_dataset'].take(1):
        break
    
    # Process a few examples
    for example_idx in range(min(sample_count, tf.shape(source_batch)[0])):
        # Get single example
        source_example = tf.expand_dims(source_batch[example_idx], 0)
        
        # Translate
        translations, attention_weights = model.translate(source_example)
        
        # Extract source sequence
        source_chars = []
        for i in range(tf.shape(source_example)[1]):
            idx = source_example[0, i].numpy()
            if idx in data_loaders['idx_to_roman'] and data_loaders['idx_to_roman'][idx] not in ['<pad>', '<sos>', '<eos>']:
                source_chars.append(data_loaders['idx_to_roman'][idx])
        
        # Extract prediction
        prediction_chars = []
        for i in range(tf.shape(translations)[1]):
            idx = translations[0, i].numpy()
            if idx in data_loaders['idx_to_native'] and data_loaders['idx_to_native'][idx] not in ['<pad>', '<sos>', '<eos>']:
                prediction_chars.append(data_loaders['idx_to_native'][idx])
            if idx == data_loaders['native_to_idx']['<eos>']:
                break
        
        # Create visualization
        if len(prediction_chars) > 0 and len(source_chars) > 0:
            attention_matrix = attention_weights[0, :len(prediction_chars), :len(source_chars)].numpy()
            
            plt.figure(figsize=(12, 8))
            sns.heatmap(attention_matrix, cmap='viridis', cbar=True)
            plt.xticks(np.arange(len(source_chars)) + 0.5, source_chars, rotation=90)
            plt.yticks(np.arange(len(prediction_chars)) + 0.5, prediction_chars)
            plt.title(f"Character Mapping: {''.join(source_chars)} → {''.join(prediction_chars)}")
            plt.tight_layout()
            plt.savefig(f"{output_dir}/mapping_{example_idx}.png", dpi=300)
            plt.close()
    
    print(f"Character mapping visualizations saved to {output_dir}")


In [11]:
# Initialize wandb

sweep_config = {
        'method': 'random',
        'metric': {
            'name': 'val_accuracy',
            'goal': 'maximize'
        },
        'parameters': {
            'cell_type': {
                'values': ['RNN', 'LSTM', 'GRU']
            },
            'embed_size': {
                'values': [32,64, 128, 256]
            },
            'hidden_size': {
                'values': [64,128, 256, 512, 1024]
            },
            'encoder_layers': {
                'values': [1, 2,3]
            },
            'dropout': {
                'values': [0.1, 0.2, 0.3]
            },
            'batch_size': {
                'values': [32, 64, 128]
            },
            'learning_rate': {
                'values': [0.001, 0.0005, 0.0001]
            },
            'teacher_forcing_ratio': {
                'values': [0.3, 0.5, 0.7,0.9]
            },
            'attention_type': {
                'values': ['bahdanau', 'dot']
            },
            'clip': {
            'values': [1.0, 5.0]
        },
            'epochs': {
                'values': [5, 10, 15,20] 
            }
        }
    }
    
# Define the sweep configuration


# Define the training function for the sweep
def sweep_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # Access all hyperparameter values through wandb.config
        config = wandb.config
        
        # Create directories
        os.makedirs('models', exist_ok=True)
        os.makedirs('results', exist_ok=True)
        
        # Prepare data
        data_loaders = prepare_data_pipeline('/kaggle/working/dakshina_dataset_v1.0/hi/lexicons/', batch_size=config.batch_size)
        
        # Initialize model
        source_vocab_size = len(data_loaders['roman_to_idx'])
        target_vocab_size = len(data_loaders['native_to_idx'])
        
        # Get special token indices
        start_token_idx = data_loaders['native_to_idx']['<sos>']
        end_token_idx = data_loaders['native_to_idx']['<eos>']
        
        model = create_model(
            source_vocab_size=source_vocab_size,
            target_vocab_size=target_vocab_size,
            embedding_dim=config.embed_size,
            hidden_units=config.hidden_size,
            cell_type=config.cell_type,
            num_layers=config.encoder_layers,
            dropout_rate=config.dropout,
            attention_type=config.attention_type,
            start_token_idx=start_token_idx,
            end_token_idx=end_token_idx
        )
        
        # Calculate model complexity
        complexity = calculate_model_complexity(
            embedding_size=config.embed_size,
            hidden_size=config.hidden_size,
            sequence_length=max(data_loaders['max_roman_len'], data_loaders['max_native_len']),
            vocabulary_size=max(source_vocab_size, target_vocab_size)
        )
        
        # Log model complexity
        wandb.log(complexity)
        
        # Create a config dictionary for training
        train_config = {
            'cell_type': config.cell_type,
            'embed_size': config.embed_size,
            'hidden_size': config.hidden_size,
            'encoder_layers': config.encoder_layers,
            'dropout': config.dropout,
            'batch_size': config.batch_size,
            'learning_rate': config.learning_rate,
            'teacher_forcing_ratio': 0.5,
            'attention_type': config.attention_type,
            'clip_norm':config.clip,
            'epochs': config.epochs  # Set a fixed number of epochs for the sweep
        }
        
        # Train model
        best_accuracy = train_model(model, data_loaders, train_config)
        
        # Evaluate model
        test_accuracy, predictions = evaluate_model(model, data_loaders)
        
        # Create attention visualizations
        create_attention_visualization_grid(predictions, sample_count=10, output_path='results/attention_grid.png')
        
        # Create character mapping visualizations
        visualize_character_mappings(model, data_loaders, sample_count=5)
        
        # Log visualizations to wandb
        wandb.log({"attention_grid": wandb.Image('results/attention_grid.png')})
        
        # Log character mapping visualizations
        for i in range(5):
            if os.path.exists(f'results/character_mappings/mapping_{i}.png'):
                wandb.log({f"character_mapping_{i}": wandb.Image(f'results/character_mappings/mapping_{i}.png')})

# Initialize the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="tf-transliteration-attention_2")


Create sweep with ID: o7dpo2cj
Sweep URL: https://wandb.ai/shivam-da24m018-iitmaana/tf-transliteration-attention_2/sweeps/o7dpo2cj


In [12]:
# Run a single experiment with default parameters
def run_single_experiment():
    # Initialize wandb
    wandb.init(project="tf-transliteration-attention", config=DEFAULT_CONFIG)
    
    # Get config
    config = wandb.config
    
    # Prepare data
    data_loaders = prepare_data_pipeline('/kaggle/working/dakshina_dataset_v1.0/hi/lexicons/', batch_size=config.batch_size)
    
    # Initialize model
    source_vocab_size = len(data_loaders['roman_to_idx'])
    target_vocab_size = len(data_loaders['native_to_idx'])
    
    # Get special token indices
    start_token_idx = data_loaders['native_to_idx']['<sos>']
    end_token_idx = data_loaders['native_to_idx']['<eos>']
    
    model = create_model(
        source_vocab_size=source_vocab_size,
        target_vocab_size=target_vocab_size,
        embedding_dim=config.embed_size,
        hidden_units=config.hidden_size,
        cell_type=config.cell_type,
        num_layers=config.encoder_layers,
        dropout_rate=config.dropout,
        attention_type=config.attention_type,
        start_token_idx=start_token_idx,
        end_token_idx=end_token_idx
    )
    
    # Calculate model complexity
    complexity = calculate_model_complexity(
        embedding_size=config.embed_size,
        hidden_size=config.hidden_size,
        sequence_length=max(data_loaders['max_roman_len'], data_loaders['max_native_len']),
        vocabulary_size=max(source_vocab_size, target_vocab_size)
    )
    
    # Log model complexity
    wandb.log(complexity)
    
    # Train model
    best_accuracy = train_model(model, data_loaders, config)
    
    # Evaluate model
    test_accuracy, predictions = evaluate_model(model, data_loaders)
    
    # Create attention visualizations
    create_attention_visualization_grid(predictions, sample_count=10, output_path='results/attention_grid.png')
    
    # Create character mapping visualizations
    visualize_character_mappings(model, data_loaders, sample_count=5)
    
    # Log visualizations to wandb
    wandb.log({"attention_grid": wandb.Image('results/attention_grid.png')})
    
    # Log character mapping visualizations
    for i in range(5):
        if os.path.exists(f'results/character_mappings/mapping_{i}.png'):
            wandb.log({f"character_mapping_{i}": wandb.Image(f'results/character_mappings/mapping_{i}.png')})
    
    wandb.finish()

# Uncomment to run a single experiment
# run_single_experiment()


In [13]:
wandb.agent(sweep_id, function=sweep_train, count=10)

wandb: Agent Starting Run: roy0zg4v with config:
wandb: 	attention_type: bahdanau
wandb: 	batch_size: 64
wandb: 	cell_type: LSTM
wandb: 	clip: 5
wandb: 	dropout: 0.2
wandb: 	embed_size: 64
wandb: 	encoder_layers: 2
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	teacher_forcing_ratio: 0.3
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250520_215431-roy0zg4v
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run decent-sweep-1
wandb: ⭐️ View project at https://wandb.ai/shivam-da24m018-iitmaana/tf-transliteration-attention_2
wandb: 🧹 View sweep at https://wandb.ai/shivam-da24m018-iitmaana/tf-transliteration-attention_2/sweeps/o7dpo2cj
wandb: 🚀 View run at https://wandb.ai/shivam-da24m018-iitmaana/tf-transliteration-attention_2/runs/roy0zg4v


Sample native script chars: ['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ']
Sample roman script chars: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


I0000 00:00:1747778073.621180      66 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747778073.622035      66 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
Epoch 1 Training:   0%|          | 0/691 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
I0000 00:00:1747778078.378691     102 cuda_dnn.cc:529] Loaded cuDNN version 90300
/

	Training Loss: 1.166
	Validation Loss: 0.891
	Accuracy: 0.0007
	New best accuracy: 0.0007, model saved


Computing accuracy: 100%|██████████| 69/69 [02:42<00:00,  2.36s/it]


...

Computing accuracy: 100%|██████████| 35/35 [02:22<00:00,  4.07s/it]


	Training Loss: 1.196
	Validation Loss: 0.988
	Accuracy: 0.0000


Computing accuracy: 100%|██████████| 35/35 [02:29<00:00,  4.28s/it]


	Training Loss: 0.942
	Validation Loss: 0.833
	Accuracy: 0.0034
	New best accuracy: 0.0034, model saved


Epoch 3 Training:  87%|████████▋ | 300/346 [10:26<07:31,  9.81s/it]